# Send Messages to Teti Kusmiati

This notebook demonstrates how to:
1. Search for Teti in WhatsApp contacts
2. Send her a lovely text message
3. Send her an audio message from a local file


In [ ]:
# Import required libraries
import requests
import json
from datetime import datetime
from typing import Dict, List, Any


In [ ]:
# Configuration
WHATSAPP_MCP_BASE_URL = "https://whatsapp-mcp-route-whatsapp-mcp.apps.rosa.akram.a1ey.p3.openshiftapps.com"
AUDIO_FILE_PATH = "/Users/akram/Downloads/message.mp3"

print(f"🔧 Configuration:")
print(f"   WhatsApp MCP Server: {WHATSAPP_MCP_BASE_URL}")
print(f"   Audio File: {AUDIO_FILE_PATH}")


In [ ]:
def test_server_connection():
    """Test the connection to the WhatsApp MCP server."""
    print("🔌 Testing server connection...")
    
    try:
        # Test health endpoint
        response = requests.get(f"{WHATSAPP_MCP_BASE_URL}/health", timeout=10)
        
        if response.status_code == 200:
            print(f"✅ Health check passed!")
            print(f"   Status: {response.text.strip()}")
            return True
        else:
            print(f"❌ Health check failed with status {response.status_code}")
            return False
            
    except requests.exceptions.RequestException as e:
        print(f"❌ Connection error: {e}")
        return False

# Test the connection
connection_ok = test_server_connection()


In [ ]:
def search_contacts(query: str) -> List[Dict[str, Any]]:
    """Search WhatsApp contacts by name or phone number."""
    print(f"🔍 Searching for contacts matching: '{query}'")
    
    try:
        # Use the HTTP API endpoint
        response = requests.post(
            f"{WHATSAPP_MCP_BASE_URL}/api/search_contacts",
            json={"query": query},
            timeout=30
        )
        
        if response.status_code == 200:
            contacts = response.json()
            print(f"✅ Found {len(contacts)} contacts")
            return contacts
        else:
            print(f"❌ Search failed with status {response.status_code}")
            print(f"   Response: {response.text}")
            return []
            
    except requests.exceptions.RequestException as e:
        print(f"❌ Search error: {e}")
        return []

# Search for Teti
contacts = search_contacts("Teti")

if contacts:
    print("\n📋 Found contacts:")
    for i, contact in enumerate(contacts, 1):
        print(f"   {i}. {contact.get('name', 'Unknown')} - {contact.get('phone_number', 'No phone')}")
else:
    print("\n❌ No contacts found for 'Teti'")


In [ ]:
def send_text_message(recipient: str, message: str) -> bool:
    """Send a text message via WhatsApp."""
    print(f"📤 Sending text message to: {recipient}")
    print(f"💬 Message: {message}")
    
    try:
        response = requests.post(
            f"{WHATSAPP_MCP_BASE_URL}/api/send_message",
            json={
                "recipient": recipient,
                "message": message
            },
            timeout=30
        )
        
        if response.status_code == 200:
            result = response.json()
            print(f"✅ Text message sent successfully!")
            print(f"   Result: {result}")
            return True
        else:
            print(f"❌ Text message failed with status {response.status_code}")
            print(f"   Response: {response.text}")
            return False
            
    except requests.exceptions.RequestException as e:
        print(f"❌ Text message error: {e}")
        return False

# Send lovely text message to Teti
if contacts:
    # Use the first contact found (assuming it's Teti)
    teti_contact = contacts[0]
    teti_phone = teti_contact.get('phone_number')
    
    if teti_phone:
        lovely_message = "Hello Teti! 👋 Hope you're having a wonderful day! This message was sent using WhatsApp MCP integration. Sending you lots of positive vibes! 🌟✨"
        
        text_success = send_text_message(teti_phone, lovely_message)
    else:
        print("❌ No phone number found for Teti")
        text_success = False
else:
    print("❌ No contacts found, cannot send message")
    text_success = False


In [ ]:
def send_audio_message(recipient: str, audio_file_path: str) -> bool:
    """Send an audio message via WhatsApp."""
    print(f"🎵 Sending audio message to: {recipient}")
    print(f"📁 Audio file: {audio_file_path}")
    
    # Check if file exists
    import os
    if not os.path.exists(audio_file_path):
        print(f"❌ Audio file not found: {audio_file_path}")
        return False
    
    try:
        # Send audio file
        with open(audio_file_path, 'rb') as audio_file:
            files = {'file': audio_file}
            data = {'recipient': recipient}
            
            response = requests.post(
                f"{WHATSAPP_MCP_BASE_URL}/api/send_audio_message",
                files=files,
                data=data,
                timeout=60  # Longer timeout for file upload
            )
        
        if response.status_code == 200:
            result = response.json()
            print(f"✅ Audio message sent successfully!")
            print(f"   Result: {result}")
            return True
        else:
            print(f"❌ Audio message failed with status {response.status_code}")
            print(f"   Response: {response.text}")
            return False
            
    except requests.exceptions.RequestException as e:
        print(f"❌ Audio message error: {e}")
        return False
    except FileNotFoundError:
        print(f"❌ Audio file not found: {audio_file_path}")
        return False

# Send audio message to Teti
if contacts and 'teti_phone' in locals():
    audio_success = send_audio_message(teti_phone, AUDIO_FILE_PATH)
else:
    print("❌ No contacts found or phone number not available, cannot send audio")
    audio_success = False


In [ ]:
# Summary of results
print("\n" + "="*50)
print("📊 SUMMARY")
print("="*50)

print(f"🔌 Server Connection: {'✅ Success' if connection_ok else '❌ Failed'}")
print(f"🔍 Contact Search: {'✅ Found ' + str(len(contacts)) + ' contacts' if contacts else '❌ No contacts found'}")

if contacts:
    teti_contact = contacts[0]
    print(f"👤 Teti Contact: {teti_contact.get('name', 'Unknown')} - {teti_contact.get('phone_number', 'No phone')}")

print(f"💬 Text Message: {'✅ Sent' if 'text_success' in locals() and text_success else '❌ Failed'}")
print(f"🎵 Audio Message: {'✅ Sent' if 'audio_success' in locals() and audio_success else '❌ Failed'}")

print("\n🎉 Mission completed! Check Teti's WhatsApp for the messages! 💌")
